In [1]:
import pandas as pd
import numpy as np
import re as re

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
validate = pd.read_csv('gender_submission.csv')

In [2]:
# Begin to Look at the Features. This is based on https://www.kaggle.com/sinakhorami/titanic-best-working-classifier
# Not a solution, just a guide on how to engineer and clean up data to boost accuracy without overfitting

# Mapping Sex
train['Sex_binary'] = train['Sex'].map({'male': 1, 'female': 0})
test['Sex_binary'] = test['Sex'].map({'male': 1, 'female': 0})

In [3]:
# Building the size of the family
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_binary,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,1
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,1,1
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,1,1
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,1,5
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,0,3
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,0,2


In [4]:
# Now what matters, if the person was alone or not
train['IsAlone'] = 0
train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1
test['IsAlone'] = 0
test.loc[test['FamilySize'] == 1, 'IsAlone'] = 1

In [5]:
# Filling empty cells with the most repeated shore
train['Embarked'] = train['Embarked'].fillna('S')
test['Embarked'] = test['Embarked'].fillna('S')

# Mapping shores
train['Embarked'] = train['Embarked'].map({'S':2,'Q':1,'C':0})
test['Embarked'] = test['Embarked'].map({'S':2,'Q':1,'C':0})

In [6]:
# Filling empty cells with median fare
train['Fare'] = train['Fare'].fillna(train['Fare'].median())
test['Fare'] = train['Fare'].fillna(train['Fare'].median())

# Splitting into 4 categories to generalize
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)

# Mapping the categories 
train.loc[train['Fare'] <= 7.91, 'Fare'] = 0
train.loc[(train['Fare'] > 7.91) & (train['Fare'] <= 14.454), 'Fare'] = 1
train.loc[(train['Fare'] > 14.454) & (train['Fare'] <= 31), 'Fare'] = 2
train.loc[train['Fare'] > 31, 'Fare'] = 3
train['Fare'] = train['Fare'].astype(int)

test.loc[train['Fare'] <= 7.91, 'Fare'] = 0
test.loc[(train['Fare'] > 7.91) & (test['Fare'] <= 14.454), 'Fare'] = 1
test.loc[(train['Fare'] > 14.454) & (test['Fare'] <= 31), 'Fare'] = 2
test.loc[train['Fare'] > 31, 'Fare'] = 3
test['Fare'] = test['Fare'].astype(int)

In [7]:
# Filling (a lot) of empty age cells with a random number between the mean - std and mean + std
ageMean = train['Age'].mean()
ageStd = train['Age'].std()

train['Age'] = train['Age'].fillna(np.random.randint(ageMean - ageStd, ageMean + ageStd))
test['Age'] = test['Age'].fillna(np.random.randint(ageMean - ageStd, ageMean + ageStd))

# Splitting into 5 categories to generalize
train['CategoricalAge'] = pd.cut(train['Age'], 5)

# Mapping the categories
train.loc[ train['Age'] <= 16, 'Age'] = 0
train.loc[(train['Age'] > 16) & (train['Age'] <= 32), 'Age'] = 1
train.loc[(train['Age'] > 32) & (train['Age'] <= 48), 'Age'] = 2
train.loc[(train['Age'] > 48) & (train['Age'] <= 64), 'Age'] = 3
train.loc[ train['Age'] > 64, 'Age'] 

test.loc[ test['Age'] <= 16, 'Age'] = 0
test.loc[(test['Age'] > 16) & (test['Age'] <= 32), 'Age'] = 1
test.loc[(test['Age'] > 32) & (test['Age'] <= 48), 'Age'] = 2
test.loc[(test['Age'] > 48) & (test['Age'] <= 64), 'Age'] = 3
test.loc[test['Age'] > 64, 'Age'] 

81    67.0
96    76.0
Name: Age, dtype: float64

In [8]:
# Getting titles of the people aboard (func from the same source listed above)
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

train['Title'] = train['Name'].apply(get_title)
test['Title'] = test['Name'].apply(get_title)
    
train['Title'] = train['Title'].replace(['Lady','Countess','Capt', 'Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'],'Rare')
train['Title'] = train['Title'].replace('Mlle', 'Miss')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')

test['Title'] = test['Title'].replace(['Lady','Countess','Capt', 'Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'],'Rare')
test['Title'] = test['Title'].replace('Mlle', 'Miss')
test['Title'] = test['Title'].replace('Ms', 'Miss')
test['Title'] = test['Title'].replace('Mme', 'Mrs')

# Mapping titles
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
train['Title'] = train['Title'].map(title_mapping)
train['Title'] = train['Title'].fillna(0)

test['Title'] = test['Title'].map(title_mapping)
test['Title'] = test['Title'].fillna(0)

In [9]:
dropElements = ['PassengerId','Name','Sex','SibSp','Parch','Ticket','Cabin','FamilySize','CategoricalFare','CategoricalAge']
train = train.drop(dropElements, axis=1)

In [10]:
features = ['Pclass','Age','Fare','Embarked','Sex_binary','IsAlone','Title']
target = 'Survived'
train[features].head(3)

,Pclass,Age,Fare,Embarked,Sex_binary,IsAlone,Title
0,3,1.0,0,2,1,0,1
1,1,2.0,3,0,0,0,3
2,3,1.0,1,2,0,1,2


In [11]:
X_train = np.array(train[features])

In [12]:
y_train = np.array(train[target])
y_train = y_train.reshape(-1,1)

In [13]:
# Import keras modules
from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import glorot_normal
from keras.optimizers import SGD

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
model = Sequential()
numNeurons = 2
model.add(Dense(9, input_dim=len(features), activation='relu', kernel_initializer="uniform")) #Si cambiamos esto dos, tenemos 76 
model.add(Dense(9, input_dim=len(features), activation='relu', kernel_initializer="uniform"))
model.add(Dense(5, activation='relu', kernel_initializer="uniform"))
model.add(Dense(1, activation='sigmoid', kernel_initializer="uniform"))

# Compiling the ANN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [15]:
model.fit(X_train, y_train, batch_size = 32, epochs = 2000)

Epoch 1/2000
891/891 [==============================] - 0s 436us/step - loss: 0.6917 - acc: 0.6061
Epoch 2/2000
891/891 [==============================] - 0s 66us/step - loss: 0.6888 - acc: 0.6162
Epoch 3/2000
891/891 [==============================] - 0s 62us/step - loss: 0.6847 - acc: 0.6162
Epoch 4/2000
891/891 [==============================] - 0s 71us/step - loss: 0.6747 - acc: 0.6162
Epoch 5/2000
891/891 [==============================] - 0s 56us/step - loss: 0.6521 - acc: 0.6162
Epoch 6/2000
891/891 [==============================] - 0s 59us/step - loss: 0.6168 - acc: 0.6162
Epoch 7/2000
891/891 [==============================] - 0s 61us/step - loss: 0.5875 - acc: 0.6162
Epoch 8/2000
891/891 [==============================] - 0s 69us/step - loss: 0.5688 - acc: 0.6162
Epoch 9/2000
891/891 [==============================] - 0s 58us/step - loss: 0.5566 - acc: 0.6162
Epoch 10/2000
891/891 [==============================] - 0s 57us/step - loss: 0.5475 - acc: 0.6162
Epoch 11/2000
891/

891/891 [==============================] - 0s 50us/step - loss: 0.4705 - acc: 0.8036
Epoch 84/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4705 - acc: 0.8114
Epoch 85/2000
891/891 [==============================] - 0s 55us/step - loss: 0.4688 - acc: 0.8013
Epoch 86/2000
891/891 [==============================] - 0s 48us/step - loss: 0.4685 - acc: 0.8013
Epoch 87/2000
891/891 [==============================] - 0s 51us/step - loss: 0.4671 - acc: 0.8025
Epoch 88/2000
891/891 [==============================] - 0s 51us/step - loss: 0.4666 - acc: 0.8058
Epoch 89/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4668 - acc: 0.8036
Epoch 90/2000
891/891 [==============================] - 0s 59us/step - loss: 0.4666 - acc: 0.7991
Epoch 91/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4666 - acc: 0.8103
Epoch 92/2000
891/891 [==============================] - 0s 48us/step - loss: 0.4686 - acc: 0.8013
Epoch 93/2000
891/891 [=

891/891 [==============================] - 0s 51us/step - loss: 0.4597 - acc: 0.8036
Epoch 166/2000
891/891 [==============================] - 0s 50us/step - loss: 0.4586 - acc: 0.8025
Epoch 167/2000
891/891 [==============================] - 0s 56us/step - loss: 0.4604 - acc: 0.8036
Epoch 168/2000
891/891 [==============================] - 0s 54us/step - loss: 0.4591 - acc: 0.8036
Epoch 169/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4589 - acc: 0.8070
Epoch 170/2000
891/891 [==============================] - 0s 55us/step - loss: 0.4601 - acc: 0.8058
Epoch 171/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4585 - acc: 0.8092
Epoch 172/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4597 - acc: 0.8103
Epoch 173/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4589 - acc: 0.8047
Epoch 174/2000
891/891 [==============================] - 0s 54us/step - loss: 0.4588 - acc: 0.8025
Epoch 175/2000


891/891 [==============================] - 0s 50us/step - loss: 0.4573 - acc: 0.8047
Epoch 248/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4582 - acc: 0.8070
Epoch 249/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4575 - acc: 0.8047
Epoch 250/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4575 - acc: 0.8047
Epoch 251/2000
891/891 [==============================] - 0s 62us/step - loss: 0.4575 - acc: 0.8025
Epoch 252/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4574 - acc: 0.8036
Epoch 253/2000
891/891 [==============================] - 0s 51us/step - loss: 0.4578 - acc: 0.8036
Epoch 254/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4581 - acc: 0.8047
Epoch 255/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4606 - acc: 0.8047
Epoch 256/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4587 - acc: 0.8081
Epoch 257/2000


891/891 [==============================] - 0s 65us/step - loss: 0.4525 - acc: 0.7991
Epoch 329/2000
891/891 [==============================] - 0s 67us/step - loss: 0.4526 - acc: 0.7957
Epoch 330/2000
891/891 [==============================] - 0s 60us/step - loss: 0.4516 - acc: 0.7991
Epoch 331/2000
891/891 [==============================] - 0s 47us/step - loss: 0.4536 - acc: 0.7935
Epoch 332/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4523 - acc: 0.8025
Epoch 333/2000
891/891 [==============================] - 0s 51us/step - loss: 0.4518 - acc: 0.7980
Epoch 334/2000
891/891 [==============================] - 0s 50us/step - loss: 0.4517 - acc: 0.7980
Epoch 335/2000
891/891 [==============================] - 0s 50us/step - loss: 0.4521 - acc: 0.7980
Epoch 336/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4537 - acc: 0.8025
Epoch 337/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4518 - acc: 0.7991
Epoch 338/2000


...

891/891 [==============================] - 0s 60us/step - loss: 0.4474 - acc: 0.7845
Epoch 1632/2000
891/891 [==============================] - 0s 58us/step - loss: 0.4475 - acc: 0.7901
Epoch 1633/2000
891/891 [==============================] - 0s 60us/step - loss: 0.4482 - acc: 0.7800
Epoch 1634/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4481 - acc: 0.7912
Epoch 1635/2000
891/891 [==============================] - 0s 50us/step - loss: 0.4491 - acc: 0.7778
Epoch 1636/2000
891/891 [==============================] - 0s 58us/step - loss: 0.4480 - acc: 0.7969
Epoch 1637/2000
891/891 [==============================] - 0s 51us/step - loss: 0.4481 - acc: 0.7856
Epoch 1638/2000
891/891 [==============================] - 0s 60us/step - loss: 0.4476 - acc: 0.7868
Epoch 1639/2000
891/891 [==============================] - 0s 59us/step - loss: 0.4481 - acc: 0.8025
Epoch 1640/2000
891/891 [==============================] - 0s 60us/step - loss: 0.4486 - acc: 0.7778
Epoch 

891/891 [==============================] - 0s 49us/step - loss: 0.4492 - acc: 0.7823
Epoch 1713/2000
891/891 [==============================] - 0s 51us/step - loss: 0.4482 - acc: 0.7834
Epoch 1714/2000
891/891 [==============================] - 0s 55us/step - loss: 0.4479 - acc: 0.7890
Epoch 1715/2000
891/891 [==============================] - 0s 51us/step - loss: 0.4490 - acc: 0.7935
Epoch 1716/2000
891/891 [==============================] - 0s 50us/step - loss: 0.4475 - acc: 0.7890
Epoch 1717/2000
891/891 [==============================] - 0s 51us/step - loss: 0.4479 - acc: 0.7901
Epoch 1718/2000
891/891 [==============================] - 0s 50us/step - loss: 0.4487 - acc: 0.8025
Epoch 1719/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4474 - acc: 0.7868
Epoch 1720/2000
891/891 [==============================] - 0s 51us/step - loss: 0.4489 - acc: 0.7879
Epoch 1721/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4476 - acc: 0.7834
Epoch 

891/891 [==============================] - 0s 52us/step - loss: 0.4479 - acc: 0.7789
Epoch 1794/2000
891/891 [==============================] - 0s 45us/step - loss: 0.4477 - acc: 0.7901
Epoch 1795/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4485 - acc: 0.8047
Epoch 1796/2000
891/891 [==============================] - 0s 48us/step - loss: 0.4483 - acc: 0.7778
Epoch 1797/2000
891/891 [==============================] - 0s 54us/step - loss: 0.4483 - acc: 0.7856
Epoch 1798/2000
891/891 [==============================] - 0s 54us/step - loss: 0.4485 - acc: 0.8002
Epoch 1799/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4474 - acc: 0.7901
Epoch 1800/2000
891/891 [==============================] - 0s 50us/step - loss: 0.4479 - acc: 0.7946
Epoch 1801/2000
891/891 [==============================] - 0s 49us/step - loss: 0.4487 - acc: 0.7901
Epoch 1802/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4490 - acc: 0.7755
Epoch 

891/891 [==============================] - 0s 52us/step - loss: 0.4478 - acc: 0.7946
Epoch 1874/2000
891/891 [==============================] - 0s 46us/step - loss: 0.4482 - acc: 0.7868
Epoch 1875/2000
891/891 [==============================] - 0s 55us/step - loss: 0.4477 - acc: 0.7924
Epoch 1876/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4483 - acc: 0.7946
Epoch 1877/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4480 - acc: 0.7811
Epoch 1878/2000
891/891 [==============================] - 0s 51us/step - loss: 0.4476 - acc: 0.7901
Epoch 1879/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4471 - acc: 0.8002
Epoch 1880/2000
891/891 [==============================] - 0s 51us/step - loss: 0.4484 - acc: 0.7811
Epoch 1881/2000
891/891 [==============================] - 0s 52us/step - loss: 0.4483 - acc: 0.7834
Epoch 1882/2000
891/891 [==============================] - 0s 51us/step - loss: 0.4477 - acc: 0.8002
Epoch 

891/891 [==============================] - 0s 52us/step - loss: 0.4455 - acc: 0.7879
Epoch 1955/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4455 - acc: 0.8025
Epoch 1956/2000
891/891 [==============================] - 0s 55us/step - loss: 0.4453 - acc: 0.8058
Epoch 1957/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4454 - acc: 0.7991
Epoch 1958/2000
891/891 [==============================] - 0s 48us/step - loss: 0.4457 - acc: 0.7901
Epoch 1959/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4453 - acc: 0.8047
Epoch 1960/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4445 - acc: 0.8047
Epoch 1961/2000
891/891 [==============================] - 0s 53us/step - loss: 0.4454 - acc: 0.7969
Epoch 1962/2000
891/891 [==============================] - 0s 54us/step - loss: 0.4451 - acc: 0.8002
Epoch 1963/2000
891/891 [==============================] - 0s 54us/step - loss: 0.4456 - acc: 0.8047
Epoch 

In [16]:
X_test = np.array(test[features])

In [17]:
predicciones = model.predict(X_test)

In [18]:
predicciones = predicciones.tolist()

In [19]:
pre = pd.Series(predicciones)
validate['prediccion'] = pre
validate['prediccion'] = validate['prediccion'].str.get(0)
validate

,PassengerId,Survived,prediccion
0,892,0,0.114409
1,893,0,0.625402
2,894,0,0.237414
3,895,0,0.092115
4,896,1,0.635305
5,897,0,0.095057
6,898,1,0.632833
7,899,0,0.206890
8,900,1,0.801553
9,901,0,0.101945


In [20]:
coincidencias = []
for dato in validate.prediccion:
    if dato >= 0.5:
        coincidencias.append(1)
    else:
        coincidencias.append(0)
validate['final'] = coincidencias
validate

,PassengerId,Survived,prediccion,final
0,892,0,0.114409,0
1,893,0,0.625402,1
2,894,0,0.237414,0
3,895,0,0.092115,0
4,896,1,0.635305,1
5,897,0,0.095057,0
6,898,1,0.632833,1
7,899,0,0.206890,0
8,900,1,0.801553,1
9,901,0,0.101945,0


In [21]:
coincide = 0
coincide = sum(validate['Survived'] == validate['final'])
print(coincide)
print(float(coincide)/ float(len(validate)))


match = 0
nomatch = 0
for val in validate.values:
    if val[1] == val[3]:
        match = match +1
    else:
        nomatch = nomatch +1
print(float(match)/float(len(validate)))

373
0.8923444976076556
0.8923444976076556


In [22]:
toKaggle = pd.DataFrame({'PassengerId':validate['PassengerId'],
                         'Survived':validate['final']})

toKaggle.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# output file with your prediction

In [23]:
from datetime import datetime
archivo = 'TitanicPred.csv'

toKaggle.to_csv(archivo,index=False)

print('Creado: ' + archivo)

Creado: TitanicPred.csv
